Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [1]:
import requests, json, threading, sys, traceback
import pandas as pd

# Funções

In [2]:
def get_category(category: str) -> list:
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    payload = {
        "partner": "linx",
        "page": 1,
        "resultsPerPage": 300,
        "multiCategory": category,
        "sortBy": "relevance",
        "department": "ecom",
        "storeId": 461,
        "customerPlus": True
    }
    
    try:
        # Pega o número total de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        max_produtos = response.json()['totalProducts']
        payload['resultsPerPage'] = max_produtos

        # Pega a lista de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        lista_p = list(response.json()['products'])
        lista_cod = []

        for prod in lista_p:
            lista_cod.append(prod['id'])
            
    except:
        return []
    
    else:
        return lista_cod

In [3]:
def get_product(code: int, verbose: int = 0) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        atributos = get_dictkey(informacoes, 'attributeGroups')
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}',
        'preco': get_dictkey(informacoes, 'currentPrice'),
        'marca': get_dictcode(atributos, 'Marca'),
        'alturaProduto': get_dictcode(atributos, 'alturaProduto'),
        'larguraProduto': get_dictcode(atributos, 'larguraProduto'),
        'pesoBruto': get_dictcode(atributos, 'pesoBruto'),
        'unidadePesoBruto': get_dictcode(atributos, 'unidadePesoBruto'),
        'profundidadeProduto': get_dictcode(atributos, 'profundidadeProduto')
        }
        
        assert(
            resultados['id'] == code and 
            resultados['preco'] != None and
            resultados['alturaProduto'] != None and 
            resultados['larguraProduto'] != None and 
            resultados['pesoBruto'] != None and 
            resultados['unidadePesoBruto'] != None and
            resultados['profundidadeProduto'] != None
        )

    except:
        if verbose >= 1:
            exc_info = sys.exc_info()
            e = ''.join(traceback.format_exception(*exc_info))
            del exc_info
            
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
        
        if verbose >= 2:     
            if 'informacoes' in locals():
                print(json.dumps(locals()['informacoes'], indent=2))
                
            if 'informacoes2' in locals():
                print(json.dumps(locals()['informacoes2'], indent=2))
        
        return None
    
    else:
        return resultados

In [4]:
def fetch_products(codigos, verbose: int = 0):
    resultados = []
    threads = []
    lock = threading.Lock()

    def thread_function(codigo, verbose):
        info = get_product(codigo, verbose)
        
        if info:
            with lock:
                resultados.append(info)

    for codigo in codigos:
        thread = threading.Thread(target=thread_function, args=(codigo, verbose,))
        threads.append(thread)
        thread.start()

    while any(thread.is_alive() for thread in threads): pass

    return resultados


In [5]:
def get_dictkey(dicionario, chave):
    # Função interna recursiva
    def busca_recursiva(d):
        if isinstance(d, dict):
            for k, v in d.items():
                if k == chave:
                    return v
                resultado = busca_recursiva(v)
                if resultado is not None:
                    return resultado
                
        elif isinstance(d, list):
            for item in d:
                resultado = busca_recursiva(item)
                if resultado is not None:
                    return resultado
        return None

    return busca_recursiva(dicionario)

In [6]:
def get_dictcode(d, code):
    # Se d for um dicionário, iteramos sobre suas chaves e valores
    if isinstance(d, dict):
        if d.get('code') == code:
            return d.get('value')  # Retorna o valor se a chave 'code' for encontrada com o valor especificado
        for value in d.values():
            result = get_dictcode(value, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    # Se d for uma lista, iteramos sobre os elementos da lista
    elif isinstance(d, list):
        for item in d:
            result = get_dictcode(item, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    return None  # Retorna None se o dicionário não for encontrado

In [7]:
def format_desc(x):
    for rep in ['<b>', '</b>', '<br>', '</br>', '<p>', '</p>', '\r', '\n', r'"""¿']:
        x = x.replace(rep, '')
    
    return x

# Constants

In [8]:
CATEGORIAS = [
    'alimentos', 
    'beleza-e-perfumaria',
    'bebidas', 
    'bebidas-alcoolicas-', 
    'limpeza', 
    # 'bebes-e-criancas', 
    'cuidados-pessoais', 
    # 'suplementos-alimentares', 
    # 'eventos-e-festas',
    # 'utensilios-e-descartaveis',
    # 'petshop', 
    # 'esporte-e-lazer',
    # 'casa-e-construcao',
    # 'floricultura-e-jardim'
]

# Code

In [9]:
map = []

for category in CATEGORIAS:
    print(f"Obtendo códigos de produtos da categoria {category}...")
    codigos = get_category(category)
    
    print(f"Obtendo informações dos produtos da categoria {category}...")
    # map[category] = fetch_products(codigos)
    for item in fetch_products(codigos, 0):
        map.append({**item, **{'categoria': category}})

map

Obtendo códigos de produtos da categoria alimentos...
Obtendo informações dos produtos da categoria alimentos...
Obtendo códigos de produtos da categoria beleza-e-perfumaria...
Obtendo informações dos produtos da categoria beleza-e-perfumaria...
Obtendo códigos de produtos da categoria bebidas...
Obtendo informações dos produtos da categoria bebidas...
Obtendo códigos de produtos da categoria bebidas-alcoolicas-...
Obtendo informações dos produtos da categoria bebidas-alcoolicas-...
Obtendo códigos de produtos da categoria limpeza...
Obtendo informações dos produtos da categoria limpeza...
Obtendo códigos de produtos da categoria cuidados-pessoais...
Obtendo informações dos produtos da categoria cuidados-pessoais...


[{'id': 74215,
  'nome': 'Açúcar Refinado UNIÃO Pacote 1kg',
  'sku': '0248334',
  'descricao': '<b>Ingredientes:</b><br>Origem vegetal: sacarose de cana de açúcar.<br><b>NÃO CONTÉM GLÚTEN.</b>',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/542/458542.jpg',
  'preco': 5.09,
  'marca': 'União',
  'alturaProduto': '24.5',
  'larguraProduto': '15.5',
  'pesoBruto': '1.009',
  'unidadePesoBruto': 'KG',
  'profundidadeProduto': '4',
  'categoria': 'alimentos'},
 {'id': 34660,
  'nome': 'Brócolis Ninja QUALITÁ 300g',
  'sku': '3007808',
  'descricao': 'Fonte de fibras, contém ferro, potássio, cálcio e magnésio. Também possui vitaminas A, C, E e K.<br><b>Dica de consumo:</b>\r\nConsumo como saladas, sopas, cremes, composição de pratos. <br><b><br><b>NÃO CONTÉM GLÚTEN.</b>',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/386/24783386.jpg',
  'preco': 9.9,
  'marca': 'Qualitá',
  'alturaProduto': '10.3',
  'larguraProduto': '15.3',
  'pesoBruto': '412',
  'unidadePesoBru

In [10]:
df = pd.concat([pd.DataFrame([item]) for item in map], ignore_index=True, sort=False, axis='index')
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,unidadePesoBruto,profundidadeProduto,categoria
0,74215,Açúcar Refinado UNIÃO Pacote 1kg,0248334,<b>Ingredientes:</b><br>Origem vegetal: sacaro...,https://static.paodeacucar.com/img/uploads/1/5...,5.09,União,24.5,15.5,1.009,KG,4,alimentos
1,34660,Brócolis Ninja QUALITÁ 300g,3007808,"Fonte de fibras, contém ferro, potássio, cálci...",https://static.paodeacucar.com/img/uploads/1/3...,9.90,Qualitá,10.3,15.3,412,G,15.6,alimentos
2,120523,Ovos Branco Extra QUALITÁ Bandeja com 20 Unidades,1510409,"Fonte de nutrientes como proteína, minerais, v...",https://static.paodeacucar.com/img/uploads/1/4...,18.90,Qualitá,7.4,29.1,1,KG,28.4,alimentos
3,334112,Requeijão Cremoso Tradicional Catupiry Copo 200g,1096020,,https://static.paodeacucar.com/img/uploads/1/2...,9.99,Catupiry,10.9,7.7,218,G,7.7,alimentos
4,655414,Iogurte Parcialmente Desnatado Morango Danone ...,1281652,"O iogurte líquido 1,25KG no sabor morango é pr...",https://static.paodeacucar.com/img/uploads/1/8...,13.99,Danone,24.4,9.9,1.291,KG,9.9,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,302930,Sabonete Líquido Dove Delicious Care Karité e ...,1046457,"""¿ O Sabonete Líquido Dove Karité e Baunilha p...",https://static.paodeacucar.com/img/uploads/1/5...,13.99,Dove,18.9,7.6,294,G,3.49,cuidados-pessoais
439,409356,NIVEA Sabonete em Barra Aveia 85g,1147219,O que é?\r\n\r\nNIVEA Sabonete em Barra Aveia ...,https://static.paodeacucar.com/img/uploads/1/8...,3.99,Nivea,5.5,9,88.5,G,3,cuidados-pessoais
440,172235,Papel Higiênico Folha Dupla Neutro QUALITÁ Pac...,6802448,<b>Composição:</b><br>Fibras celulósicas.<br><...,https://static.paodeacucar.com/img/uploads/1/5...,7.99,Qualitá,20.2,21.1,360.9,G,11.1,cuidados-pessoais
441,182417,Desodorante Aerosol GIOVANNA BABY Rosa 150ml,4731924,,https://static.paodeacucar.com/img/uploads/1/4...,14.29,Giovanna Baby,18,5,188,G,5,cuidados-pessoais


In [11]:
df['descricao'] = df['descricao'].apply(format_desc)
df['descricao'] = df['descricao'].fillna('Sem descrição')
df['pesoBruto'] = df['pesoBruto'].astype(float)
df['pesoBruto'] = df.apply(lambda x: x['pesoBruto'] if x['unidadePesoBruto'] == 'G' else x['pesoBruto'] * 1000, axis=1)
df.drop(columns=['unidadePesoBruto'], inplace=True)
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,profundidadeProduto,categoria
0,74215,Açúcar Refinado UNIÃO Pacote 1kg,0248334,Ingredientes:Origem vegetal: sacarose de cana ...,https://static.paodeacucar.com/img/uploads/1/5...,5.09,União,24.5,15.5,1009.0,4,alimentos
1,34660,Brócolis Ninja QUALITÁ 300g,3007808,"Fonte de fibras, contém ferro, potássio, cálci...",https://static.paodeacucar.com/img/uploads/1/3...,9.90,Qualitá,10.3,15.3,412.0,15.6,alimentos
2,120523,Ovos Branco Extra QUALITÁ Bandeja com 20 Unidades,1510409,"Fonte de nutrientes como proteína, minerais, v...",https://static.paodeacucar.com/img/uploads/1/4...,18.90,Qualitá,7.4,29.1,1000.0,28.4,alimentos
3,334112,Requeijão Cremoso Tradicional Catupiry Copo 200g,1096020,,https://static.paodeacucar.com/img/uploads/1/2...,9.99,Catupiry,10.9,7.7,218.0,7.7,alimentos
4,655414,Iogurte Parcialmente Desnatado Morango Danone ...,1281652,"O iogurte líquido 1,25KG no sabor morango é pr...",https://static.paodeacucar.com/img/uploads/1/8...,13.99,Danone,24.4,9.9,1291.0,9.9,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...
438,302930,Sabonete Líquido Dove Delicious Care Karité e ...,1046457,"""¿ O Sabonete Líquido Dove Karité e Baunilha p...",https://static.paodeacucar.com/img/uploads/1/5...,13.99,Dove,18.9,7.6,294.0,3.49,cuidados-pessoais
439,409356,NIVEA Sabonete em Barra Aveia 85g,1147219,O que é?NIVEA Sabonete em Barra Aveia é um sab...,https://static.paodeacucar.com/img/uploads/1/8...,3.99,Nivea,5.5,9,88.5,3,cuidados-pessoais
440,172235,Papel Higiênico Folha Dupla Neutro QUALITÁ Pac...,6802448,Composição:Fibras celulósicas.Informação Adici...,https://static.paodeacucar.com/img/uploads/1/5...,7.99,Qualitá,20.2,21.1,360.9,11.1,cuidados-pessoais
441,182417,Desodorante Aerosol GIOVANNA BABY Rosa 150ml,4731924,,https://static.paodeacucar.com/img/uploads/1/4...,14.29,Giovanna Baby,18,5,188.0,5,cuidados-pessoais


In [12]:
PATH_TO_SAVE = '..\data\produtos.csv'

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Arklok\AppData\Local\Temp\ipykernel_23064\3786087928.py:1: SyntaxWarning: invalid escape sequence '\d'
  PATH_TO_SAVE = '..\data\produtos.csv'


In [14]:
old_df = pd.read_csv(PATH_TO_SAVE, sep='|', encoding='utf-8')

In [15]:
df = df[~df['id'].isin(old_df['id'])]

In [16]:
df = pd.concat([old_df, df], ignore_index=True, sort=False, axis='index')

C:\Users\Arklok\AppData\Local\Temp\ipykernel_23064\1987210897.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([old_df, df], ignore_index=True, sort=False, axis='index')


In [17]:
df.to_csv(r'..\data\produtos.csv', index=False, sep='|', encoding='utf-8')